In [2]:
from sqlalchemy import create_engine
import quandl
import pandas as pd
import numpy as np
import talib as ta
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score
from datetime import datetime as dt, timedelta

## SQL Connection

In [3]:
def create_connection():
    # Login details: 
    DSN_Name = 'traderDSN' ; Login_ID = 'basic' ; pwd = 'pwd1'

    # The engine does the same job as a connection and a cursor
    engine = create_engine(r'mssql+pyodbc://'+Login_ID+':'+pwd+'@'+ DSN_Name) 

    return engine 

In [4]:
# Create engine and query 
engine = create_connection()
query = ''' 
SELECT * FROM {schema}.{table_name}
'''.format(schema='stocks', table_name='Adjusted_Fortune_500')

# Importing a Pandas Dataframe
prices = pd.read_sql(query,engine)
prices = prices.drop(columns=['index','close', 'split coefficient','dividend amount'])
prices = prices.rename(columns={'Ticker':'symbol','adjusted close':'close','Name':'name'})

## Format Data

In [5]:
# Reformat column types
prices['open']  = prices['open'].astype(float)
prices['high']  = prices['high'].astype(float)
prices['low']   = prices['low'].astype(float)
prices['close'] = prices['close'].astype(float)
prices['volume']= prices['volume'].astype(float)
prices['date'] = pd.to_datetime(prices['date'])

# Create and sort indexes
prices.set_index(['name', 'symbol', 'date'], inplace=True)
prices = prices.sort_index(level=1, ascending=True)

In [6]:
# Calculated columns
prices  = prices.copy().loc[(prices.loc[:,'open':'volume'] != 0 ).all(axis=1)] # Don't touch this (:

# Group over the symbols. 
prices_g = prices.groupby(level=['name'])

# Calculate many moving averages, of closing prices
EMA_lengths = [20, 50, 200]  
for l in EMA_lengths: 
    prices['EMA_'+str(l)] = prices_g.close.apply(lambda x: ta.EMA(x,l))   

prices['EMA_ratio'] = prices['EMA_200']/prices['EMA_50'] # Ratio of moving average
prices['RSI'] = prices_g.close.apply(lambda x: ta.RSI(x, 14))  # 14 day relative strength index

def CCI_func(x):
    xh = x.high 
    xl = x.low 
    xc = x.close 
    return ta.CCI(high=xh, low=xl, close=xc, timeperiod=5)

prices['CCI'] = prices_g.apply(CCI_func).reset_index(level=0,).drop(columns='name')    # created an extra column. 

prices['Momentum'] = prices_g.close.apply(lambda x: ta.MOM(x, 10))  # 10 day rolling momentum changed to prices_g.close

def Stoch_fast(x, index):
    xh = x.high 
    xl = x.low 
    xc = x.close 
    return ta.STOCHF(high=xh, low=xl, close=xc, fastk_period=14, fastd_period=3)[index] # Params 14, 3    

#prices['Stoch_fastk'] = prices_g.apply(Stoch_fast, 0).reset_index(level=1,).drop(columns='symbol')   # Stochastic fast indicator
#prices['Stoch_fastd'] = prices_g.apply(Stoch_fast, 1).reset_index(level=1,).drop(columns='symbol')

# def OBV_func(x):
#    xc = x.close
#     xv = x.volume
#    return ta.OBV(xc, xv) 
#prices['OBV'] = prices_g.apply(OBV_func).reset_index(level=1,).drop(columns='symbol')  # On Balance Volume


def bband_func(x, index):
    xc = x.close
    return ta.BBANDS(xc)[index]

prices['bbandupper'] = prices_g.apply(bband_func, 0).reset_index(level=0,).drop(columns='name')
prices['bbandmiddle'] = prices_g.apply(bband_func, 1).reset_index(level=0,).drop(columns='name') 
prices['bbandlower'] =  prices_g.apply(bband_func, 2).reset_index(level=0,).drop(columns='name')

# Standadises bands
prices['bbandlowerii'] =  prices['bbandlower']/prices['close']
prices['bbandupperii'] =  prices['bbandupper']/prices['close']

# Create target column
prices['5_day_target'] = prices.groupby(['name'])['close'].transform(lambda x:x.shift(-5))
prices['perc_change']=prices['5_day_target']/prices['close']


In [7]:
# Import treasury yield data
US_yields=quandl.get("USTREASURY/YIELD", authtoken="wdLjKc5NXiuwUhFLByKm")
US_yields=US_yields[['3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '30 YR']]
US_yields.index.rename('date', inplace=True)

In [8]:
# Join prices and treasury data
prices.reset_index(level='date',inplace=True)
test=pd.merge(prices, US_yields, how='left', left_on='date' , right_index=True)
test.set_index('date',append=True,inplace=True)
test.sort_index(level=1, ascending=True,inplace=True)

In [9]:
# Remove non-calculated data
merged = test.copy().loc[(test.loc[:,'open':'30 YR'] != 0 ).all(axis=1)]
merged.drop(columns=['30 YR'], inplace=True)
merged.fillna(method='pad', inplace=True)
clean_prices = merged.loc[(merged['EMA_200'].notnull() & merged['5_day_target'].notnull())]

In [10]:
# Reorganise columns
clean_prices=clean_prices.loc[:,[
'open',
'high',
'low',
'close',
'volume',
'EMA_20',
'EMA_50',
'EMA_200',
'bbandupper',
'bbandmiddle',
'bbandlower',
'EMA_ratio',
'RSI',
'CCI',
'Momentum',
#'Stoch_fastk',
#'Stoch_fastd',
#'OBV',
'bbandlowerii',
'bbandupperii',
'3 MO',
'6 MO',
'1 YR',
'2 YR',
'3 YR',
'5 YR',
'7 YR',
'10 YR',
'5_day_target',
'perc_change']]

## Machine Learning

In [11]:
# Set test size per stock
test_size = 3

In [12]:
# Split features and target
X = clean_prices.copy().loc[:,'EMA_ratio':'10 YR'] 
y = clean_prices.copy().loc[:, ['perc_change']]

In [13]:
# Get series of ingroup rownumbers
X['num']= X.groupby(level=1).cumcount(ascending=False)

# Use num to split train/test by date
X_train = X.loc[X.num > test_size ].drop(columns='num')
X_test =  X.loc[X.num <= test_size ].drop(columns='num')
y_train = y.loc[X.num > test_size ]
y_test =  y.loc[X.num <= test_size ]

In [14]:
# Train machine learning
forest = RandomForestRegressor()
forest.fit(X_train,y_train)

C:\Users\Student\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [15]:
# Create test results
y_pred = forest.predict(X_test)
y_pred = pd.DataFrame({'ans': y_pred})

In [16]:
# R-squared value
r2_score(y_test,y_pred)

0.282156106552661

## Making Predictions

In [17]:
# Extract current data
predict_days=clean_prices.sort_index(level=1, ascending=True).groupby(['name']).tail(1).loc[:,'EMA_ratio':'10 YR']

In [18]:
predict_days['5dayforecast'] = forest.predict(predict_days)

In [19]:
result=predict_days.join(merged, lsuffix='l',how ='left')[['close','5dayforecast']]

In [20]:
result['5day_price']=result['close']*result['5dayforecast']

In [21]:
result['5day%_change']=(result['5dayforecast']-1)*100

## May be useful in the future

In [23]:
prep = result[['close','5day_price','5day%_change']]
prep.sort_values("5day%_change",ascending=False, inplace=True)
prep.reset_index(inplace=True)
prep.set_index('name')
tdf=timedelta(days=5)
prep=prep[prep['date']>=(dt.now()-tdf)]

C:\Users\Student\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Pickling for Dash

In [31]:
prep.to_pickle('top_movers.pkl')
clean_prices.to_pickle('clean_prices.pkl')

In [32]:
clean_prices.head()

open   high    low  close     volume     EMA_20  \
name  symbol date                                                           
Alcoa AA     2017-08-17  39.00  39.00  37.75  37.80  3272834.0  37.276650   
             2017-08-18  37.88  38.21  37.34  37.76  2028802.0  37.322683   
             2017-08-21  38.19  38.66  38.05  38.47  2331799.0  37.431952   
             2017-08-22  38.74  40.13  38.73  39.77  3867097.0  37.654623   
             2017-08-23  39.56  41.44  39.45  41.06  4820314.0  37.978945   

                            EMA_50    EMA_200  bbandupper  bbandmiddle  \
name  symbol date                                                        
Alcoa AA     2017-08-17  35.724955  33.258206   39.302113       37.900   
             2017-08-18  35.804761  33.303000   39.299573       37.896   
             2017-08-21  35.909280  33.354413   39.406075       38.152   
             2017-08-22  36.060681  33.418250   40.187928       38.606   
             2017-08-23  36.256733  33.494287   41.516182       38.972   

                            ...       3 MO  6 MO  1 YR  2 YR  3 YR  5 YR  \
name  symbol date           ...                                            
Alcoa AA     2017-08-17     ...       1.00  1.11  1.24  1.32  1.46  1.76   
             2017-08-18     ...       1.02  1.13  1.24  1.33  1.47  1.77   
             2017-08-21     ...       1.00  1.11  1.23  1.32  1.46  1.76   
             2017-08-22     ...       1.00  1.13  1.24  1.33  1.48  1.80   
             2017-08-23     ...       1.00  1.11  1.22  1.32  1.45  1.76   

                         7 YR  10 YR  5_day_target  perc_change  
name  symbol date                                                
Alcoa AA     2017-08-17  2.01   2.19         41.34     1.093651  
             2017-08-18  2.01   2.19         41.21     1.091367  
             2017-08-21  2.00   2.18         42.17     1.096179  
             2017-08-22  2.04   2.22         43.00     1.081217  
             2017-08-23  1.99   2.17         43.09     1.049440  

[5 rows x 27 columns]